# Export Training

In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
from random import *
import glob
import os


frame = pd.DataFrame()
list_ = []

In [2]:
path =r'C:\Users\Vasileas\Desktop\Test'
allFiles = glob.glob(path + "/*.csv")

for file_ in allFiles:
    
    df = pd.read_csv(file_,index_col=None, header=0,error_bad_lines=False)
    
    df=df[['Div','Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG','HTR','B365H','B365D','B365A']]
    df.columns=['LeagueDivision','MatchDate','HomeTeam','AwayTeam','FullTimeHomeTeamGoals','FullTimeAwayTeamGoals'
                ,'FullTimeResultID','HalfTimeHomeTeamGoals','HalfTimeAwayTeamGoals','HalfTimeResult'
               ,'Bet365HomeWinOdds','Bet365DrawOdds','Bet365AwayWinOdds']

    df=df.replace(to_replace=['D','H','A'], value=[0,1,2])
    
    #Relative odds 
    df['RelativeHomeAway']=df['Bet365HomeWinOdds']-df['Bet365AwayWinOdds']
    df['RelativeHomeDraw']=df['Bet365HomeWinOdds']-df['Bet365DrawOdds']
    df['RelativeAwayDraw']=df['Bet365AwayWinOdds']-df['Bet365DrawOdds']
    
    #Γκανιότα
    df["Ganiota"]=np.nan
    for index,row in df.iterrows():
        if (row["Bet365HomeWinOdds"]!=0 and row["Bet365AwayWinOdds"]!=0 and row["Bet365DrawOdds"]!=0):
            row["Ganiota"]=float(1)/row["Bet365HomeWinOdds"]+float(1)/row["Bet365AwayWinOdds"]+float(1)/row["Bet365DrawOdds"]-1   
            df.ix[index]=row
    
    #Probabilities of Booker
    df["ProbaHome"]=0
    df["ProbaAway"]=0
    df["ProbaDraw"]=0

    for index,row in df.iterrows():
        row["ProbaHome"]=float(1)/row["Bet365HomeWinOdds"]
        row["ProbaAway"]=float(1)/row['Bet365AwayWinOdds']
        row["ProbaDraw"]=float(1)/row['Bet365DrawOdds']
        df.ix[index]=row

    df["PureProbaHome"]=df["ProbaHome"]-df['Ganiota']/3
    df["PureProbaAway"]=df["ProbaAway"]-df['Ganiota']/3
    df["PureProbaDraw"]=df["ProbaDraw"]-df['Ganiota']/3
    
    #Match-dates-as-timestamp
    df["NewMatchDate"]=np.nan

    for index,row in df.iterrows():
        row["NewMatchDate"]=pd.to_datetime(row[["MatchDate"]].values[0])
        df.ix[index]=row 

    #Encoding DIV 
    from sklearn.preprocessing import LabelEncoder
    encoding=LabelEncoder()
    encoding.fit(df["LeagueDivision"].values)

    league_div=encoding.transform(df["LeagueDivision"].values) 

    df["EncodingLeagueDivision"]=league_div

    #Result of the last game 

    df["HomeLastRes"] = np.nan
    df["VisitorLastRes"]=np.nan

    from collections import defaultdict
    res_last=defaultdict(int) 


    for index,row in df.iterrows(): 
        home_team=row["HomeTeam"]    
        away_team=row["AwayTeam"] 
        row['HomeLastRes']=res_last[home_team] 
        row['VisitorLastRes']=res_last[away_team]
        df.ix[index]=row 
        res_last[home_team]=row["FullTimeResultID"] 
        res_last[away_team]=row["FullTimeResultID"]

    #Win streak

    df["HomeWinStreak"] = 0
    df["AwayWinStreak"]= 0
    win_streak=defaultdict(int)

    for index,row in df.iterrows(): 
        home_team=row["HomeTeam"]    
        away_team=row["AwayTeam"] 
        row["HomeWinStreak"]=win_streak[home_team] 
        row["AwayWinStreak"]=win_streak[away_team]
        df.ix[index]=row
        if(row["FullTimeResultID"]==1):
            win_streak[home_team]=win_streak[home_team]+1
            win_streak[away_team]=0
        elif(row["FullTimeResultID"]==2) :
            win_streak[home_team]=0
            win_streak[away_team]=win_streak[away_team]+1
        else :
            win_streak[away_team]=0
            win_streak[home_team]=0
            
    #Points Until The end of season
    df['HomeTeamPoints'] = 0
    df['AwayTeamPoints']=0

    #MaximumPointsInLeague,MinimumPointsInLeague

    df['HowMuchMaximumTeamPointsInLeague'] = 0
    df['HowMuchMinimumTeamPointsInLeague'] = 0

    maxleague=0
    minleague=0

    points=defaultdict(int)


    for index,row in df.iterrows(): 
        home_team=row["HomeTeam"]    
        away_team=row["AwayTeam"] 

        row["HomeTeamPoints"]=points[home_team] 
        row['AwayTeamPoints']=points[away_team] 
        row['HowMuchMaximumTeamPointsInLeague'] = max([i for i in points.values()])
        row['HowMuchMinimumTeamPointsInLeague'] = min([i for i in points.values()])

        df.ix[index]=row
        if(row["FullTimeResultID"]==1):
            points[home_team]=points[home_team]+3

        elif(row["FullTimeResultID"]==2) :
            points[away_team]=points[away_team]+3

        else :
            points[home_team]=points[home_team]+1
            points[away_team]=points[away_team]+1
            
    #Points As Host Points As Away.+ Goals
    df['HomeTeamPointsAsHost'] = 0
    df['AwayTeamPointsAsAway'] = 0

    df['HomeTeamGoalsAsHost'] = 0
    df['AwayTeamGoalsAsAway'] = 0

    pointshost=defaultdict(int)
    pointsaway=defaultdict(int)

    goalshost=defaultdict(int)
    goalsaway=defaultdict(int)

    for index,row in df.iterrows(): 
        home_team=row["HomeTeam"]    
        away_team=row["AwayTeam"] 

        row['HomeTeamPointsAsHost']=pointshost[home_team] 
        row['AwayTeamPointsAsAway']=pointsaway[away_team] 

        row['HomeTeamGoalsAsHost']=goalshost[home_team]
        row['AwayTeamGoalsAsAway']=goalsaway[away_team]

        df.ix[index]=row
        if(row["FullTimeResultID"]==1):
            pointshost[home_team]=pointshost[home_team]+3

        elif(row["FullTimeResultID"]==2) :
            pointsaway[away_team]=pointsaway[away_team]+3

        else :
            pointshost[home_team]=pointshost[home_team]+1
            pointsaway[away_team]=pointsaway[away_team]+1

        goalshost[home_team]=goalshost[home_team]+row['FullTimeHomeTeamGoals']
        goalsaway[away_team]=goalsaway[away_team]+row['FullTimeAwayTeamGoals']


    #Relative points and goals
    df['RelativeTeamPointsAsHostAsAway'] = df['HomeTeamPointsAsHost'] - df['AwayTeamPointsAsAway'] 
    df['RelativeTeamGoalsAsHostAsAway']  = df['HomeTeamGoalsAsHost'] - df['AwayTeamGoalsAsAway'] 
    
    #Relative points
    df['RelativeHomeTeamPoints']=np.nan
    df['RelativeAwayTeamPoints']=np.nan
    for index,row in df.iterrows():
        if (row['HowMuchMaximumTeamPointsInLeague'] !=0):
            row['RelativeHomeTeamPoints']=float(row['HomeTeamPoints'])/row['HowMuchMaximumTeamPointsInLeague']
            row['RelativeAwayTeamPoints']=float(row['AwayTeamPoints'])/row['HowMuchMaximumTeamPointsInLeague']
            df.ix[index]=row
    #Relative PointsDifference
    df["RelativePointsDifference"]=df['RelativeHomeTeamPoints']-df['RelativeAwayTeamPoints']

    #GOAL Που έχει δώσει συνολικα και έχει φάει η καθε ομάδα
    df['SumHomeTeamGoals'] = 0
    df['SumAwayTeamGoals'] = 0

    df['EatSumHomeTeamGoals'] = 0
    df['EatSumAwayTeamGoals'] = 0
    #γκολ δώσει
    goals=defaultdict(int)

    #Γκόλ φάει 
    goalseat=defaultdict(int)

    for index,row in df.iterrows(): 
        home_team=row["HomeTeam"]    
        away_team=row["AwayTeam"] 

        row["SumHomeTeamGoals"]=goals[home_team] 
        row['SumAwayTeamGoals']=goals[away_team] 
        row['EatSumHomeTeamGoals'] = goalseat[home_team] 
        row['EatSumAwayTeamGoals']= goalseat[away_team] 

        df.ix[index]=row

        goals[home_team]=goals[home_team] + row['FullTimeHomeTeamGoals']
        goals[away_team]=goals[away_team] + row['FullTimeAwayTeamGoals']      
        goalseat[home_team]=goalseat[home_team]+ row['FullTimeAwayTeamGoals']
        goalseat[away_team] = goalseat[away_team] +row['FullTimeHomeTeamGoals']


    #Relative Goals, Relative EatGoals

    df['RelativeSumTeamGoals'] = df['SumHomeTeamGoals'] - df['SumAwayTeamGoals']
    df['RelativeEatSumTeamGoals'] = df['EatSumHomeTeamGoals'] - df['EatSumAwayTeamGoals']

    #PointsDifference

    df['PointsDifference'] =0

    df['PointsDifference'] =df['HomeTeamPoints'] - df['AwayTeamPoints']
 
    #Day+Month
    df['Month']=np.nan
    df['Day']=np.nan
    for index,row in df.iterrows():
        row["Month"]=row[["NewMatchDate"]].dt.month
        row["Day"]=row[["NewMatchDate"]].dt.day
        df.ix[index]=row
    
    list_.append(df)

C:\Users\Vasileas\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
C:\Users\Vasileas\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:259: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]


In [3]:
frame = pd.concat(list_)
frame.to_csv('FullTest.csv', index=False)
print('Done')

Done
